In [ ]:
import cupy as cp
import numpy as np

aA = np.array([1,2,3])
caA = cp.asarray(aA)
fLen = cp.linalg.norm(caA)

print(fLen)



In [ ]:
import os
os.environ["OPENCV_IO_ENABLE_OPENEXR"] = "1"
import cv2

import numpy as np
from pathlib import Path

pathMain = Path(r"[main path]")
pathLocalPos3d = pathMain / "AT_LocalPos3d_Raw"
pathObjectIdx = pathMain / "AT_ObjectIdx_Raw"
pathObjectLoc3d = pathMain / "AT_ObjectLoc3d_Raw"
pathLabel = pathMain / "AT_Label/full_res/SemSeg"

sFrame1:str = "Frame_0010.exr"
sFrame2:str = "Frame_0011.exr"

def LoadImages(_sFrame:str):
    pathImgLocalPos3d = pathLocalPos3d / _sFrame
    pathImgObjectIdx = pathObjectIdx / _sFrame
    pathImgObjectLoc3d = pathObjectLoc3d / _sFrame

    imgLocalPos3d = cv2.imread(
        pathImgLocalPos3d.as_posix(),
        cv2.IMREAD_ANYCOLOR | cv2.IMREAD_ANYDEPTH | cv2.IMREAD_UNCHANGED,
    )

    imgObjectIdx = cv2.imread(
        pathImgObjectIdx.as_posix(),
        cv2.IMREAD_ANYCOLOR | cv2.IMREAD_ANYDEPTH | cv2.IMREAD_UNCHANGED,
    )

    imgObjectLoc3d = cv2.imread(
        pathImgObjectLoc3d.as_posix(),
        cv2.IMREAD_ANYCOLOR | cv2.IMREAD_ANYDEPTH | cv2.IMREAD_UNCHANGED,
    )

    return imgLocalPos3d, imgObjectIdx, imgObjectLoc3d
# enddef

imgPos1, imgObj1, imgLoc1 = LoadImages(sFrame1)
imgPos2, imgObj2, imgLoc2 = LoadImages(sFrame2)

aOffset = np.array([[[1e4, 1e4, 1e4]]])
imgPos1 = imgPos1 - aOffset
imgPos2 = imgPos2 - aOffset

imgLoc1 = imgLoc1 - aOffset
imgLoc2 = imgLoc2 - aOffset

iChX: int = 2
iChY: int = 1
iChZ: int = 0

# print(np.min(imgPos1))
# print(np.max(imgPos1))


In [ ]:
from scipy import spatial
from scipy import interpolate

def EvalObjFlow(*, _iObjIdx: int, _imgPos1: np.ndarray, _imgPos2: np.ndarray, _imgMask1: np.ndarray, _imgMask2: np.ndarray):
    iRows, iCols, iChan = _imgPos1.shape

    aIdxFlat1 = np.argwhere(_imgMask1)
    aPosFlat1 = _imgPos1[_imgMask1, :].reshape(-1, 3)
    aPosFlat2 = _imgPos2[_imgMask2, :].reshape(-1, 3)

    print("> Evaluating Delaunay triangulation of object local position values. This may take a while...")
    xTri2 = spatial.Delaunay(aPosFlat2)

    print("> Interpolating positions from image 1 to image 2...")
    aMeshPixX, aMeshPixY = np.meshgrid(np.arange(0, iCols), np.arange(0, iRows))

    # Flatten mesh pixel coordinates and select only those of valid pixels
    aMask2 = _imgMask2.flatten()
    aPixX = np.ndarray.flatten(aMeshPixX)[aMask2]
    aPixY = np.ndarray.flatten(aMeshPixY)[aMask2]

    # Calculate interpolators
    ipolX = interpolate.LinearNDInterpolator(xTri2, aPixX)
    ipolY = interpolate.LinearNDInterpolator(xTri2, aPixY)

    # Apply interpolation for norm colors of A to obtain mapped
    # pixel coordinates for image A in image B.
    aPixMapX = ipolX(aPosFlat1)
    aPixMapY = ipolY(aPosFlat1)

    print("> Generating flow image...")
    aPixMapXY = np.stack((aPixMapX, aPixMapY), axis=1)

    aPixMapMask = np.all(np.logical_not(np.isnan(aPixMapXY)), axis=1)

    aPixXY = aIdxFlat1[aPixMapMask][:,::-1]
    aPixMapXY = aPixMapXY[aPixMapMask]

    # aFlowLines = np.stack((aPixXY, aPixMapXY), axis=1)
    aFlowXY = aPixMapXY - aPixXY

    aIdxFlatMap1 = aIdxFlat1[aPixMapMask]
    aIdxPos = aIdxFlatMap1[:,0] * iCols + aIdxFlatMap1[:,1]

    imgValidMap = np.zeros((iRows, iCols), dtype=bool)
    aValidMapFlat = imgValidMap.flatten()
    aValidMapFlat[aIdxPos] = True
    imgValidMap = aValidMapFlat.reshape(iRows, iCols)

    imgFlow = np.zeros((iRows, iCols, 4), dtype=np.float32)
    imgFlow[imgValidMap, iChX] = aFlowXY[:,0]
    imgFlow[imgValidMap, iChY] = aFlowXY[:,1]
    imgFlow[imgValidMap, iChZ] = _iObjIdx
    imgFlow[imgValidMap, 3] = 1.0

    return imgFlow, imgValidMap
# enddef


def EvalFlow(*, _imgPos1: np.ndarray, _imgPos2: np.ndarray, _imgObj1: np.ndarray, _imgObj2: np.ndarray):

    # Find set of unique objects
    print("Find set of unique objects...")
    iRows, iCols, iChan = _imgObj1.shape
    
    imgObjFlat1 = _imgObj1.reshape(-1, iChan)
    aU, aMaskObjIdx1 = np.unique(imgObjFlat1, axis=0, return_inverse=True)

    imgMaskObjIdx1 = aMaskObjIdx1.reshape(iRows, iCols)
    
    iObjCnt = aU.shape[0]
    print(f"Found {iObjCnt} unique objects")

    imgFlow = np.zeros((iRows, iCols, 4), dtype=np.float32)

    for iObjIdx in range(0, 1):
        print(f"\nEvaluating flow for object {iObjIdx}...")
        imgMask1 = imgMaskObjIdx1 == iObjIdx
        imgMask2 = np.all(_imgObj2 == aU[iObjIdx][np.newaxis, np.newaxis, :], axis=2)
        
        imgObjFlow, imgObjValid = EvalObjFlow(_iObjIdx=iObjIdx, _imgMask1=imgMask1, _imgMask2=imgMask2, _imgPos1=_imgPos1, _imgPos2=_imgPos2)
        imgFlow[imgObjValid] = imgObjFlow[imgObjValid]
    # endfor

    print("\nWriting flow image...")
    pathFlow = pathMain / "Flow.exr"
    cv2.imwrite(pathFlow.as_posix(), imgFlow)
    print("done!")

# enddef

In [ ]:
# EvalFlow(_imgPos1=imgPos1, _imgPos2=imgPos2, _imgObj1=imgObj1, _imgObj2=imgObj2)


In [ ]:
iRows, iCols, iChan = imgObj1.shape
imgObjFlat1 = imgObj1.reshape(-1, iChan)
aU, aMaskObjIdx1 = np.unique(imgObjFlat1, axis=0, return_inverse=True)
imgMaskObjIdx1 = aMaskObjIdx1.reshape(iRows, iCols)


In [ ]:
aObjFlat2 = imgObj2.reshape(-1, iChan)
aObjIdxFlat2 = np.ones((iRows * iCols), dtype=int) * -1

for iObjIdx in range(len(aU)):
    aObjIdxFlat2[np.all(aObjFlat2 == aU[iObjIdx], axis=1)] = iObjIdx
# endfor

imgMaskObjIdx2 = aObjIdxFlat2.reshape(iRows, iCols)


In [ ]:
aInvalidIdx = np.argwhere(aU[:,2] == 0).reshape(-1)
for iIdx in aInvalidIdx:
    imgMaskObjIdx1[imgMaskObjIdx1 == iIdx] = -1
    imgMaskObjIdx2[imgMaskObjIdx2 == iIdx] = -1
# endfor


In [ ]:
# Option "axis" of unique() not implemented, yet.
cimgObjFlat1 = cp.asarray(imgObjFlat1)
caU, caMaskObjIdx1 = cp.unique(imgObjFlat1, axis=0, return_inverse=True)
cimgMaskObjIdx = caMaskObjIdx1.reshape(iRows, iCols)


In [ ]:
imgObjFlat2 = imgObj2.reshape(-1, iChan)
aU2, aMaskObjIdx2 = np.unique(imgObjFlat2, axis=0, return_inverse=True)
imgMaskObjIdx2 = aMaskObjIdx2.reshape(iRows, iCols)


In [ ]:
import matplotlib.pyplot as plt
from scipy import spatial
from scipy import interpolate

iObjIdx = 2
# plt.imshow(imgInv)
imgMask1 = imgMaskObjIdx1 == iObjIdx
imgObjPos1 = np.zeros((iRows, iCols, 4), dtype=float)
imgObjPos1[imgMask1, 0:3] = imgPos1[imgMask1, :]
imgObjPos1[imgMask1, 3] = 1.0
plt.imshow(imgObjPos1)


In [ ]:

imgMask2 = np.all(imgObj2 == aU[iObjIdx][np.newaxis, np.newaxis, :], axis=2)
imgObjPos2 = np.zeros((iRows, iCols, 4), dtype=float)
imgObjPos2[imgMask2, 0:3] = imgPos2[imgMask2, :]
imgObjPos2[imgMask2, 3] = 1.0
plt.imshow(imgObjPos2)


In [ ]:
print(cp.cuda.runtime.getDeviceCount())


In [ ]:
# cfdImage = cp.cuda.texture.ChannelFormatDescriptor(32, 32, 32, 0, cp.cuda.runtime.cudaChannelFormatKindFloat)
import sys
from pathlib import Path

pathKernel = Path.cwd() / "Dev-EvalFlow-v3.cu"
sKernelCode = pathKernel.read_text()

iThreadCnt = 32
tiSearchRadiusXY = (200, 200)

iPosRows, iPosCols, iPosChanCnt = imgPos1.shape
tiSizeXY = (iPosCols, iPosRows)
tiStartXY = (3250, 1250)
tiRangeXY = (500, 500)

# Full image
tiStartXY = (0, 0)
tiRangeXY = (iPosCols, iPosRows)

tiRangeXY = tuple(tiRangeXY[i] if tiStartXY[i] + tiRangeXY[i] <= tiSizeXY[i] else tiSizeXY[i] - tiStartXY[0] for i in range(2))

tiBlockDimXY = (tiRangeXY[0] // iThreadCnt + (1 if tiRangeXY[0] % iThreadCnt > 0 else 0), tiRangeXY[1])

iIdxCols = tiRangeXY[0]
iIdxRows = tiRangeXY[1]
iIdxChan = 5

iPosRowStride = iPosCols * iPosChanCnt
iIdxRowStride = iIdxChan * iIdxCols

iSubPixChanCnt = 4
iSubPixRowStride = iSubPixChanCnt * iIdxCols

sFuncFlowExp = (f"EvalFlow<{tiStartXY[0]}, {tiStartXY[1]}, "
                f"{tiRangeXY[0]}, {tiRangeXY[1]}, "
                f"{tiSizeXY[0]}, {tiSizeXY[1]}, "
                f"{tiSearchRadiusXY[0]}, {tiSearchRadiusXY[1]}, "
                f"{iPosChanCnt}, {iPosRowStride}, "
                f"{iIdxChan}, {iIdxRowStride}, "
                f"{iSubPixChanCnt}, {iSubPixRowStride}>")
                
modKernel = cp.RawModule(code=sKernelCode, options=("-std=c++11",), name_expressions=[sFuncFlowExp])
# modKernel.compile(log_stream=sys.stdout)
kernFlow = modKernel.get_function(sFuncFlowExp)

caIdxMapXY = cp.ones((iIdxRows, iIdxCols, iIdxChan), dtype=cp.int32)
caIdxMapXY *= -1

caSubPixMapXY = cp.full((iIdxRows, iIdxCols, iSubPixChanCnt), cp.nan, dtype=cp.float32)

caPos1 = cp.asarray(imgPos1, dtype=cp.float32)
caPos2 = cp.asarray(imgPos2, dtype=cp.float32)
caMaskObjIdx1 = cp.asarray(imgMaskObjIdx1, dtype=cp.int32)
caMaskObjIdx2 = cp.asarray(imgMaskObjIdx2, dtype=cp.int32)

kernFlow(tiBlockDimXY, (iThreadCnt,), (caPos1, caPos2, caMaskObjIdx1, caMaskObjIdx2, caIdxMapXY, caSubPixMapXY))

# cp.cuda.Stream.null.synchronize()

aIdxMapXY = cp.asnumpy(caIdxMapXY)
aSubPixMapXY = cp.asnumpy(caSubPixMapXY)


In [ ]:
iSrcRow = 1343
iSrcCol = 3250
aSrcPos = imgPos1[iSrcRow, iSrcCol]
iObjIdx1 = imgMaskObjIdx1[iSrcRow, iSrcCol]
print(f"aSrcPos: {aSrcPos}")

iMapRow = iSrcRow - tiStartXY[1]
iMapCol = iSrcCol - tiStartXY[0]
print(f"tiSizeXY: {tiSizeXY}")
print(f"tiStartXY: {tiStartXY}")
print(f"iSrcCol, iSrcRow: {iSrcCol}, {iSrcRow}")
print(f"iMapCol, iMapRow: {iMapCol}, {iMapRow}")

aMapXY = aIdxMapXY[iMapRow, iMapCol]
print(f"aMapXY: {aMapXY}")

iRow = aMapXY[1]
iCol = aMapXY[0]
aOptPos = imgPos2[iRow, iCol]
iObjIdx2 = imgMaskObjIdx2[iRow, iCol]
fDist = np.linalg.norm(aSrcPos - aOptPos)
fDist *= fDist
print(f"(id1, id2), (iX, iY), fDist, aOptPos: ({iObjIdx1}, {iObjIdx2}), ({iCol}, {iRow}): {fDist}, {aOptPos}")
print("")

iRow0 = aMapXY[1]
iCol0 = aMapXY[0]

for j in range(-1, 2):
    iRow = iRow0 + j
    for i in range(-1, 2):
        iCol = iCol0 + i
        aOptPos = imgPos2[iRow, iCol]
        iObjIdx2 = imgMaskObjIdx2[iRow, iCol]
        fDist = np.linalg.norm(aSrcPos - aOptPos)
        fDist *= fDist
        print(f"(id1, id2), (iX, iY), fDist, aOptPos: ({iObjIdx1}, {iObjIdx2}), ({iCol}, {iRow}): {fDist}, {aOptPos}")
    # endfor
# endfor



In [ ]:
iSelObjIdx = 2
# Select object id and those points that have a valid map
aMaskObjIdx = np.logical_and(aIdxMapXY[:, :, 0] == iSelObjIdx, aIdxMapXY[:, :, 3] >= 0)

aFlatIdxXY = aIdxMapXY[aMaskObjIdx][:, 1:3].astype(float)
aFlatMapIdxXY = aSubPixMapXY[aMaskObjIdx][:, 0:2]
aFlowLines = np.stack((aFlatIdxXY, aFlatMapIdxXY), axis=1)

# Plot flow image
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection

clnFlow = LineCollection(aFlowLines[::1], linewidths=1.0, cmap="jet", alpha=0.5)

figFlow, axFlow = plt.subplots()
# axFlow.imshow(imgObjPos1, alpha=0.5)
# axFlow.imshow(imgObjPos2, alpha=0.5)

axFlow.add_collection(clnFlow)
axFlow.set_xlim(max(0, tiStartXY[0] - tiSearchRadiusXY[0]), min(tiStartXY[0] + tiRangeXY[0] + tiSearchRadiusXY[0], tiSizeXY[0]))
axFlow.set_ylim(min(tiStartXY[1] + tiRangeXY[1] + tiSearchRadiusXY[1], tiSizeXY[1]), max(0, tiStartXY[1] - tiSearchRadiusXY[1]))
axFlow.margins(0.1)


In [ ]:
aMaskValidFlow = aIdxMapXY[:, :, 3] >= 0
aValidIdxXY = aIdxMapXY[aMaskValidFlow][:, 1:3]
aValidIdxPos = aValidIdxXY[:,1] * iPosCols + aValidIdxXY[:, 0]

aFlowXY = aSubPixMapXY[aMaskValidFlow][:, 2:4]
aObjIdx = aIdxMapXY[aMaskValidFlow][:, 0].astype(float)

imgValidMap = np.zeros((iPosRows, iPosCols), dtype=bool)
aValidMapFlat = imgValidMap.flatten()
aValidMapFlat[aValidIdxPos] = True
imgValidMap = aValidMapFlat.reshape(iPosRows, iPosCols)

imgFlow = np.zeros((iPosRows, iPosCols, 4), dtype=np.float32)
imgFlow[imgValidMap, iChX] = aFlowXY[:,0]
imgFlow[imgValidMap, iChY] = aFlowXY[:,1]
imgFlow[imgValidMap, iChZ] = aObjIdx
imgFlow[imgValidMap, 3] = 1.0

pathFlow = pathMain / "Flow.exr"
cv2.imwrite(pathFlow.as_posix(), imgFlow)


In [ ]:

imgObjIdx = imgMaskObjIdx1[tiStartXY[1]:(tiStartXY[1] + tiRangeXY[1]), tiStartXY[0]:(tiStartXY[0] + tiRangeXY[0])]

aIdxX = np.linspace(tiStartXY[0], tiStartXY[0] + tiRangeXY[0] - 1, tiRangeXY[0])
aIdxY = np.linspace(tiStartXY[1], tiStartXY[1] + tiRangeXY[1] - 1, tiRangeXY[1])
aMeshX, aMeshY = np.meshgrid(aIdxX, aIdxY)
aIdxXY = np.concatenate((aMeshX[:,:,np.newaxis], aMeshY[:,:,np.newaxis]), axis=2)

aFlatIdxXY = aIdxXY.reshape(-1, 2)
aFlatIdxMapXY = aIdxMapXY.reshape(-1, 6)

aMask = np.all(aFlatIdxMapXY >= 0, axis=1)
aFlatObjIdx = imgObjIdx.flatten()

aError = np.argwhere(aFlatObjIdx != aFlatIdxMapXY[:,3])

# print(aFlatIdxXY[aError[0]])


In [ ]:
iSelObjIdx = 0
aMaskObjIdx = np.logical_and(aFlatObjIdx == iSelObjIdx, aMask)
aFlowLines = np.stack((aFlatIdxXY[aMaskObjIdx], aFlatIdxMapXY[aMaskObjIdx, 0:2]), axis=1)

# Plot flow image
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection

clnFlow = LineCollection(aFlowLines[::100], linewidths=1.0, cmap="jet", alpha=0.5)

figFlow, axFlow = plt.subplots()
# axFlow.imshow(imgObjPos1, alpha=0.5)
# axFlow.imshow(imgObjPos2, alpha=0.5)

axFlow.add_collection(clnFlow)
axFlow.set_xlim(max(0, tiStartXY[0] - tiSearchRadiusXY[0]), min(tiStartXY[0] + tiRangeXY[0] + tiSearchRadiusXY[0], tiSizeXY[0]))
axFlow.set_ylim(min(tiStartXY[1] + tiRangeXY[1] + tiSearchRadiusXY[1], tiSizeXY[1]), max(0, tiStartXY[1] - tiSearchRadiusXY[1]))
axFlow.margins(0.1)


In [ ]:
aWndRC = np.array([50, 50])

aIdxFlat1 = np.argwhere(imgMask1)

aIdxFlatMin1 = aIdxFlat1 - aWndRC
aIdxFlatMax1 = aIdxFlat1 + aWndRC

aPosFlat1 = imgPos1[imgMask1, :].reshape(-1, 3)
aPosFlat2 = imgPos2[imgMask2, :].reshape(-1, 3)

aMinIdx = np.zeros((aPosFlat1.shape[0], 3), dtype=int)

caPosFlat2 = cp.asarray(aPosFlat2)
print(caPosFlat2.device)

caPosFlat1 = cp.asarray(aPosFlat1)
caDiff = cp.abs(caPosFlat2- caPosFlat1)
aDiff = cp.asnumpy(caDiff)


# for iIdx, aVec1 in enumerate(aPosFlat1):
#     aSum = np.sum(np.abs(aPosFlat2 - aVec1), axis=1)

    # caVec1 = cp.asarray(aVec1)
    # caDiff = cp.abs(caPosFlat2 - caVec1)
    # caSum = cp.sum(caDiff, axis=1)
    # caMinIdx = cp.argsort(caSum)[0:3]

    # aMinIdx[iIdx] = cp.asnumpy(caMinIdx)
    # aMinIdx[iIdx] = np.argsort(np.sum(np.abs(aPosFlat2 - aVec1), axis=1))[0:3]
# endfor

# aPosFlat2[aMinIdx[0]]
# for iIdx, aVec1 in enumerate(aPosFlat1):
#     aMinIdx[iIdx] = np.argmin(np.linalg.norm(aPosFlat2 - aVec1, axis=1, ord=1))
#     # break
# # endfor

In [ ]:
# Write flow image
aFlowXY = aPixMapXY - aPixXY

aIdxFlatMap1 = aIdxFlat1[aPixMapMask]
aIdxPos = aIdxFlatMap1[:,0] * iPosCols + aIdxFlatMap1[:,1]

imgValidMap = np.zeros((iPosRows, iPosCols), dtype=bool)
aValidMapFlat = imgValidMap.flatten()
aValidMapFlat[aIdxPos] = True
imgValidMap = aValidMapFlat.reshape(iPosRows, iPosCols)

# imgFlowMask1 = imgMask1.flatten()[aIdxFlatMap1].reshape(iRows, iCols)

imgFlow = np.zeros((iPosRows, iPosCols, 4), dtype=np.float32)
imgFlow[imgValidMap, iChX] = aFlowXY[:,0]
imgFlow[imgValidMap, iChY] = aFlowXY[:,1]
imgFlow[imgValidMap, iChZ] = iObjIdx
imgFlow[imgValidMap, 3] = 1.0

pathFlow = pathMain / "Flow.exr"
cv2.imwrite(pathFlow.as_posix(), imgFlow)


# plt.imshow(imgValidMap)

In [ ]:
# Plot flow image
from matplotlib.collections import LineCollection

clnFlow = LineCollection(aFlowLines[::500], linewidths=1.0, cmap="jet", alpha=0.5)

figFlow, axFlow = plt.subplots()
axFlow.imshow(imgObjPos1, alpha=0.5)
axFlow.imshow(imgObjPos2, alpha=0.5)

axFlow.add_collection(clnFlow)
axFlow.set_xlim(0, iPosCols)
axFlow.set_ylim(iPosRows, 0)
axFlow.margins(0.1)


In [ ]:

# imgObjPos2 = np.zeros((iRows, iCols, 4), dtype=float)
# imgObjPos2[imgMask1, 0:3] = imgPos1[imgMask1, :]
# imgObjPos2[imgMask1, 3] = 1.0
# plt.imshow(imgObjPos1)

# aMask = aMaskObjIdx == iObjIdx

# aPos1 = imgPos1.reshape(-1, 3)[aMask]
# aPos2 = imgPos2.reshape(-1, 3)[aMask]

# aCol = np.linspace(0, iCols - 1, iCols)
# aRow = np.linspace(0, iRows - 1, iRows)
# aMeshPixX, aMeshPixY = np.meshgrid(aX, aY)


# aMeshPixX, aMeshPixY = np.meshgrid(np.arange(0, iCols), np.arange(0, iRows))

# # Flatten mesh pixel coordinates and select only those of valid pixels
# aPixX = np.ndarray.flatten(aMeshPixX)[aMask]
# aPixY = np.ndarray.flatten(aMeshPixY)[aMask]


In [ ]:
imgDx = imgObjPos1
plt.imshow(imgDx)

In [ ]:
aCol = np.linspace(0, iPosCols - 1, iPosCols)
aRow = np.linspace(0, iPosRows - 1, iPosRows)

ipolObjPos1 = interpolate.RegularGridInterpolator((aRow, aCol), imgObjPos1)


In [ ]:
aVals = 

In [ ]:

# Triangulate image data of B
# Takes ages!
# xTri2 = spatial.Delaunay(aPos2) #, qhull_options="QJ Qbb")


In [ ]:

# Calculate interpolators
xInterX = interpolate.LinearNDInterpolator(xTri2, aPixX)
xInterY = interpolate.LinearNDInterpolator(xTri2, aPixY)

# Apply interpolation for norm colors of A to obtain mapped
# pixel coordinates for image A in image B.
aPixMapX = xInterX(aPos1)
aPixMapY = xInterY(aPos1)

# Calculate difference between mapped and original pixel positions
aPixDiffX = aPixX - aPixMapX
aPixDiffY = aPixY - aPixMapY
